In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import pandas as pd


In [9]:
df=pd.DataFrame(
{
    'a':[1,1,2,3,4],
    'b':[2,2,3,2,1],
    'c':[4,6,7,8,9],
    'd':[4,3,4,5,4]
})

In [10]:
X=df.assign(const=1)
X

,a,b,c,d,const
0,1,2,4,4,1
1,1,2,6,3,1
2,2,3,7,4,1
3,3,2,8,5,1
4,4,1,9,4,1


In [20]:
pd.Series([vif(X.values,i) for i in range(X.shape[1])],index=X.columns) # we don't care about VIF of the constants

b         1.185185
c         1.370370
d         1.185185
const    61.666667
dtype: float64

In [19]:
X.drop('a',axis=1,inplace=True)
pd.Series([vif(X.values,i) for i in range(X.shape[1])],index=X.columns)

b         1.185185
c         1.370370
d         1.185185
const    61.666667
dtype: float64

In [23]:
import numpy as np
import statsmodels.formula.api as smf

def get_vif(exogs,data):
    vif_array=np.array([])
    tolerance_array=np.array([])
    
    for exog in exogs:
        not_exog=[i for i in exogs if i!=exog]
        formula=f"{exog}~{'+'.join(not_exog)}"
        
        
        r_squared=smf.ols(formula,data=data).fit().rsquared
        
        vif=1/(1-r_squared)
        vif_array=np.append(vif_array,vif).round(2)
        
        tolerance=1-r_squared
        tolerance_array=np.append(tolerance_array,tolerance).round(2)
    df_vif= pd.DataFrame({'VIF':vif_array, 'Tolerance':tolerance_array}, index=exogs)
    
    return df_vif

In [24]:
get_vif(exogs=df.columns,data=df)

,VIF,Tolerance
a,22.95,0.04
b,3.00,0.33
c,12.95,0.08
d,3.00,0.33


# Thresholds

1. VIF if it's higher than 10, it a problem. remove the column with highest VIF and test again
2. Tolerance if it is lower than 0.1, it's a problem

In [26]:
from statsmodels.tsa.stattools import adfuller
df = pd.DataFrame([np.random.normal(10,4,1000)]).T
df.columns = ['a']
df.head()

res = adfuller(df.a)
print(res)

(-32.00587079334664, 0.0, 0, 999, {'1%': -3.4369127451400474, '5%': -2.864437475834273, '10%': -2.568312754566378}, 5590.760009690245)


In [27]:
print(f"ADF STATS:{res[0]}")
print(f"p-value: {res[1]}")
print('Critical Values:')
for i,j in res[4].items():
    print(f"\t {i}:{j}")

ADF STATS:-32.00587079334664
p-value: 0.0
Critical Values:
	 1%:-3.4369127451400474
	 5%:-2.864437475834273
	 10%:-2.568312754566378


#since p value is equal to 0, we reject the Null Hypothesis.So our data has no unit root 
#Meaning no autocorrelation

if the residuals are not normally distributed that means that we missed a significant variable into the model